<h1><center> Modeling and Simulation: Apple Orchard Revenue </center></h1>

<div>
<img src="appleOrchard.jpg",align="center", width = "600">
</div>

## Problem Statement:
An apple grower wishes to develop a simulation of her orchard's revenue in a single year. 

Suppose

**-** There are *m* trees;

**-** The number of apples produced by tree *i* is $A_i$;

**-** The $\text{j}^{th}$ apple from tree *i* is $a_{i,j}$

**-** The quality of apply *j* from tree *i* is $Q_{i,j} \in [0,1]$

**-** The weight of apple *j* from tree i is $W_{i,j}$ in grams.

**-** Apples of quality greater tha $0.9$ (premiium grade) receive a price of $\$r_1$ per kilo

**-** Apples of quality between $0.5$ and $0.9$ (first grade) receive a price of $\$r_2$

**-** Apples of quality less than $0.5$ (second grade) receive a price of $\$r_3$ per kilo.

## Tasks

### Task 1
Write down an expression for the total return from the orchard (in a single year)

#### Orchard's Total Return
Define $P \colon Q_{i,j} \to \{\$r_1,\$r_2,\$r_3 \}$

where $P$ is the function that maps from quality to price $\$r_i$.

Define $Z$ as the total return for the orchard.

Then, we have

\begin{align*} Z = \sum_i^{m}\sum_j^{A_i} \frac{P(Q_{i,j})}{1000}W_{i,j} \end{align*}



### Task 2

- Suggest a type of distribution for $A_i$


- Given that you know $A_i$, what is the distribution of the total number of premium grade apples from tree *i*, in terms of $p_1$ = $\mathbb{P}(Q_{i,j} \in [0.9,1])?$ 


- Suggest a type of distribution for $Q_{i,j}$


- Suggest a type of distribution for $W_{i,j}$



#### Distribution for *A*<sub>i</sub>



We know that $A_i \geq 0$, because $A_i$ is the number of apples produced by tree i. Thus, we restrict the distribution of $A_i$ to those with nonnegative, integer support.
As $A_i$ represents the count of the number of apples produced by tree $i$, the Poisson distribution is a natural choice. Formally, we have 
$$A_i \stackrel{iid}{\sim} Poisson(\lambda), \text{for }  \lambda \geq 0$$

Given $A_i$, the distribution of the total number of premium grade apples from tree $i$ is distributed as a Binomial random variable. Formally, if we define 
$$\begin{align} p_i & :=  \text{ # of premium grade apples from tree i}\\ 
A_i & :=  \text{ # of apples produce by tree i}\\
& \text{then,}\\
p_i|A_i & \sim Bin(A_i,p_1 = Bin(A_i, 0.1)) \end{align}$$

#### Distribution for *Q*<sub>i,j</sub>
$Q_{i,j}$ is the quality rating of the $j^{th}$ apple from the $i^{th}$ tree. Also, $Q_{i,j} \in (0,1)$.
Both a natural and flexibule choice of distribution is the Beta distribution. Fomrally, we take
$$Q_{i,j} \stackrel{iid}{\sim} Beta(\alpha,\beta)$$

#### Distribution for *W*<sub>i,j</sub>
$W_{i,j}$ is the weight of $j^{th}$ apple from the $i^{th}$ tree. And so, $W_{i,j} \geq 0$.
Although, previous research has shown that the distribution of apple weight has a Normal distribution, the constraint of $W_{i,j} \geq 0$ is obviously violated if $W_{i,j}$ is Normally distributed. 

A more pertinent distribution for apple weight is the Log-normal distribution whose justification will be given below. Formally, we have
$$W_{i,j} \stackrel{iid}{\sim} Lognormal(\mu,\sigma^2)$$

### Task 3
Write down an expression for the expected return from the orchard (in a single year)

$$\begin{align} E[Z] = & E[ \sum_i^{m}\sum_j^{A_i} \frac{P(Q_{i,j})}{1000}W_{i,j}]\\
= & \sum_i^{m}E[\sum_j^{A_i} \frac{P(Q_{i,j})}{1000}W_{i,j}] \\
= & \sum_i^{m}E[E[\sum_j^{A_i} \frac{P(Q_{i,j})}{1000}W_{i,j} |A_i]]\\
= & \sum_i^{m}E[A_i\frac{E[P(Q_{i,j})]}{1000}E[W_{i,j}]]\\
= & m\Bigg[\frac{\lambda}{1000}\big(r_1\int_{0.9}^{1}f(q)dq + r_2\int_{0.5}^{0.9}f(q)dq + r_3\int_{0}^{0.5}f(q)dq\Big)\big(e^{\mu + \frac{\sigma^2}{2}}\Big)\Bigg]\end{align}$$




#### Expected Return Widget
To explore the effects of different levels of the parameters and of price, we create a widget that calculates expected revenue from the above formula.



In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import numpy as np
import ipywidgets as widgets
import scipy.stats as stats

In [2]:
@interact(m = widgets.IntSlider(min=1,max=100,step=1,value=1),
         lam = widgets.IntSlider(min = 50, max = 400, step = 10, value = 50),
         r1 = widgets.FloatSlider(min = 0,max = 5,step = 0.5,value = 1),
         r2 =widgets.FloatSlider(min = 0,max = 5,step = 0.5,value = 1),
         r3 = widgets.FloatSlider(min = 0,max = 5,step = 0.5,value = 1),
         mu = widgets.FloatSlider(min = 2,max = 7,step = .5,value = 1),
         sigma2 = widgets.FloatSlider(min = 0.01,max = 0.05,step = 0.001,value = 0.01),
         alpha = widgets.IntSlider(min = 1,max = 100,step = 1,value = 5),
         beta = widgets.IntSlider(min = 1,max = 100,step = 1,value = 5))

def expZ(m,lam,r1,r2,r3,mu,sigma2,alpha,beta):
    result = (m*(lam)/1000)*((r1)*(stats.beta.cdf(1,alpha,beta)-stats.beta.cdf(0.9,alpha,beta))
                         + (r2)*(stats.beta.cdf(0.9,alpha,beta)-stats.beta.cdf(0.5,alpha,beta))
                         + (r3)*(stats.beta.cdf(0.5,alpha,beta)))*np.exp(mu + (sigma2/2))
    print("\n\n\tExpected Revenue: ${0:.2f}".format(result))


interactive(children=(IntSlider(value=1, description='m', min=1), IntSlider(value=50, description='lam', max=4…

## Model Realism and Assumptions

To ensure an adequate amount of realism in the modeling of apple-tree production and apple weights, we incorporate the results of three articles:

1. The Journal of the American Society for Horticultural Science by Miranda, Carlos  and Royo, J.,  [Statistical Model Estimates Potential Yields in Golden Delicious and Royal Gala Apples before Bloom(2003)](https://www.researchgate.net/publication/248701745_Statistical_Model_Estimates_Potential_Yields_in_Golden_Delicious'_and_Royal_Gala'_Apples_before_Bloom).


2. The New Zealand Journal of Crop and Horticultural Science, by Jianlu Zhang , Graham F. Thiele & Richard N. Rowe 
[Gala apple fruit (1995)](https://www.tandfonline.com/doi/pdf/10.1080/01140671.1995.9513871).


3. Hessong, Athena. (n.d.). When Do Apple Trees Set Fruit? Home Guides | SF Gate. http://homeguides.sfgate.com/apple-trees-set-fruit-58656.html



The results of article (1) that will used are the following:
1. $\text{Fruit Weight of Golden Delicious Apples (g/apple): Mean $\pm$ SD =}$ $157.4 \pm 19.7$


2. $\text{Yield per Golden Delicious Apple Tree (kg/tree): Mean $\pm$ SD =}$ $33.2 \pm 15.4$



The result of article (2) that will be used and ***modified*** are:

1. $\text{The distribution of apple weights takes the shape of a Normal distribution}$.

The result of article (3) that will be used is:

1. $\text{Fruit that has set on the tree can take between 100 and 200 days for the apples to be ready for harvest}$


### Modeling *A*<sub>i</sub>

We have already specified the number of apples produced by tree $i$, $A_i$, as a Poisson random variable with parameter $\lambda$.

We know that the $mle$ estimate of $\lambda$ from an $iid$ sample $A_1, A_2, \ldots, 
A_n$ drawn from a Poisson distribution is $\bar A = n^{-1}\sum_{i=1}^{n}A_i$.

As a proxy to the mle estimate, we calculate $\bar A$ from the mean apple weight 
and mean yield per apple tree as follows:

Define

$\bar{W} :=$ mean fruit weight of apples (g/apple)

$\bar Y : = $ mean yield per apple tree (kg/tree)

$$\begin{align} \hat \lambda = & \frac{\bar Y \text{(kg/tree)}}{\bar W \text{(g/apple)}}\\
= & \frac{1000 \bar Y \text{(g/tree)}}{\bar W \text{(g/apple)}}\\
= & 1000\frac{ \bar Y}{\bar W}\text{(apple/tree)} \approx 211 \text{(apple/tree)}\end{align}$$
 

### Modeling *W*<sub>i,j</sub>



The article, Gala apple fruit, by Zhang, Thiele, and Rowe found that the distribution of weight (g) of Gala apples was apparently Normal. This implies that the support of $W_{i,j}$, the weight of $j^{th}$ apple from the $i^{th}$ tree, is $(-\infty, \infty)$. Yet, we know all apples have non-negative weight.

Whatever growth process produces the distribution of apple weights, it can never results in an apple at maturity having negative weight. For this reason, any distribution that assigns positive probability, however small, to apple weights that are less than zero are incorrect.



#### An Exponential Growth Process for Apple Weight

Many natural growth processes are driven by the accumulation of small, apparently random, percentage changes over time. On the log scale, these changes become additive, and by the Central Limit Theorem, have a Normal distribution.

We hypothesize that the underlying growth process of an apple is a function of an exponential growth process. In fact, it seems plausible that a logistic growth process would characterize the apple's weight trajectory during its maturation, as both the maximum weight of each apple is contrained by biological and environmental factors and the rate of increase of the weight of an apple should decrease as it approaches its expected maximum weight.

For our purposes, however, it will be sufficient to model the weight of $j^{th}$ apple from the $i^{th}$ tree on day $t$, according to a stochastic exponential growth equation such that

$$\begin{align} & W_{i,j}(t) = P_0\text{ }e^{Rt}\\
\text{where, } & P_0 = 1\\
& R \sim N(\mu,\sigma^2) \end{align}$$



If we wanted a plausiable trajectory of the growth of an apple over its maturation, we might heuristically take the number of days of growth, $D$, to be 150, the average of the upper and lower bounds for the number of days for an apple to grow, given by article (3).

Next, we would need to back out $\mu$ and $\sigma^2$ for the Normal distribution of daily growth rates.

To do this, we rely upon the relationship between the $Normal$ and $Lognormal$ distributions such that:

If $X \sim N(\mu_x,\sigma^2_x)$, then $Y = e^x \sim LN(\mu_y, \sigma^2_y)$, then given $\mu_y$ and $\sigma^2_y$,


$$\begin{align} \mu_x & = 2\ln(\mu_y) - \frac{1}{2}\ln(\sigma^2_y + \mu_y^2)\\
\sigma^2_x & = -2\ln(\mu_y) + \frac{1}{2}\ln(\sigma^2_y + \mu_y^2) \end{align}$$

From article (1), we have $\hat \mu_y \approx 157$ and $\hat \sigma^2_y \approx 388$. Also, we assume that the data on which these statistics are calculated are Lognormally distributed.

Thus, from the relationship between the parameters of the $Normal$ and $Lognormal$ above, we have

$$\begin{align} \hat\mu_x & \approx 5 \\
\hat\sigma^2_x & \approx 0.04 \end{align}$$


The above values of $\hat\mu_x$ and $\hat\sigma^2_x$ are the parameters for a full season of growth, i.e., approximately 150 days.

Parameter values for daily growth would be $\frac{\hat\mu_x}{150}$ and $\frac{\hat\sigma^2_x}{150^2}$

For revenue simulation purposes, however, we will have need only of the maturity distribution of apple weight. And so, we will use $\hat\mu_x \approx 5$ and $\hat\sigma^2_x \approx 0.04$ for the parameters of the distribution of the seasonal growth for apple weight.

### Modeling *Q*<sub>i,j</sub>

We allow for flexibility in specifying the shape of the distribution of the quality of apples through giving the user the ability to change the parameter values of the $Beta$ distribution.

## Orchard Simulation Class and Visualization

Next, we create an Orchard Simulation class called OrchardSim to allow for a cleaner more tractable qualitative analysis of the relationship of orchard revenue to the random variables of apple amount, quality, and weight.

To help along intuition, we create a widget to visualize how changes to parameter values changes the distributions of revenue, apple production, apple quality, and apple weight.

### OrchardSim Class

In [3]:

## orchard_revenue.py
# Class definition for orchard revenue simulation

from __future__ import print_function
import seaborn as sns
import numpy as np



class OrchardSim:
    """
        Class used to simulate apple orchard revenues
        
        Attributes 
        --------------
        nscen (int):
            the number of scenarios to simulate
        
        trees (int):
            the number of trees in the orchard
        
        R (list):
            a list of revenue per kilogram of apples by grade
            
            R[0] premium grade revenue per kilogram
            
            R[1] first grade revenue per kilogram
            
            R[2] second grade revenue per kilogram
        
        qualityCutoff (list):
            a list of quality cutoffs
            
            qualityCutoff[0] premium grade lower bound
            
            qualityCutoff[1] first grade upper bound
            
            qualityCutoff[i] in (0,1)
        
        amt_param (int):
            the paramter for the Poisson distribution to simulate number of apples grown on a tree

        q_alpha (float):
            parmater for Beta distribution to simulate quality of apples
        
        q_beta (float):
            parameter for Beta distribution to simulate quality of apples
            
        w_mean (float):
            mean for Normal distribution to simulate apple weights
            
        w_var (float):
            variance for Normal distribution to simulate apple weights
        
        """
    @ staticmethod
    def inverseMu(ybar,s2_hat):
        t1 = 2*np.log(ybar)
        t2 = (1/2)*np.log(s2_hat + ybar**2)
        return(t1 - t2)
    @ staticmethod
    def inverseVar(ybar,s2_hat):
        t1 = np.log(s2_hat + ybar**2)
        t2 = 2*np.log(ybar)
        return(t1 - t2)

    def __init__(self,nscen,trees,amt_param,q_alpha,q_beta,w_mean,w_var,R = [2,1.5,1],qualityCutoff = [0.5,0.9]):
   
        self.nscen = nscen
        self.trees = trees
        
        # price per kilo for grade 1 apples
        self.r1 = R[0]
        # price per kilo for grade 2 apples
        self.r2 = R[1]
        # price per kilo for grade 3 apples
        self.r3 = R[2]
        self.rev_list = [self.r1,self.r2,self.r3]
        
        self.A = np.zeros([self.trees,nscen])
        self.amt_param = amt_param

        self.Q = np.zeros([self.trees,nscen])
        self.q_alpha = q_alpha
        self.q_beta = q_beta
        
        self.gprem_ub = 1
        self.gprem_lb = qualityCutoff[1]
        self.g1st_ub = qualityCutoff[1]
        self.g1st_lb = qualityCutoff[0]
        self.g2nd_ub = qualityCutoff[0]
        self.g2nd_lb = 0
       
    
        self.w_mean = self.inverseMu(w_mean,w_var)
        self.w_sd = np.sqrt(self.inverseVar(w_mean,w_var))
        
        self.rev_df = np.zeros(nscen)
    

    def priceMap(self,q):
        """
        Function to map quality of apples to prices
        
        """
        if( q >= self.gprem_lb):
            p = self.r1
            return(p)
        elif(q >= self.g1st_lb and q < self.gprem_lb):
            p = self.r2
            return(p)
        else:
            p = self.r3
            return(p)
    
    def simOrchardGrowth(self):
        """
        Function to simulate orchard growth and revenue
        """
        # Simulate number of apples produced by trees across all scenarios
        self.A = np.random.poisson(self.amt_param,[self.trees,self.nscen])
       
        # Create array of total number of apples produced in each scenario
        totalA = self.A.sum(0)
        self.Q_list = []
        self.P_list = []
        self.W_list = []
        # Vectorize priceMap function
        pMapVec = np.vectorize(self.priceMap)
   
        for scen in range(self.nscen):
            q = np.random.beta(self.q_alpha,self.q_beta,totalA[scen])
            self.Q_list.append(q)
            p = pMapVec(q)
            self.P_list.append(p)
            w = np.exp(np.random.normal(self.w_mean, self.w_sd, totalA[scen]))
            self.W_list.append(w)
            self.rev_df[scen] = (1/1000)*np.dot(w,p)
            
    def plotRevenueDistn(self,ax_other = None):
        hist_kws_dict={"histtype": "bar", "linewidth": 1.5,
                  "alpha": 1, "color": "g","edgecolor":"black"}
        ax = sns.distplot(self.rev_df,kde = False, hist_kws = hist_kws_dict)
        ax.set_title("Histogram of Orchard Revenue")
        ax.set_xlabel("Revenue ($)")
        ax.set_ylabel("Count")
        ax_other = ax
        return ax_other

        
    
    def plotQualityDistn(self,ax_other = None):
        hist_kws_dict={"histtype": "bar", "linewidth": 1.5,
                  "alpha": 1, "color": "g","edgecolor":"black"}
        y = np.hstack(self.Q_list)
        ax = sns.distplot(y,kde = False, hist_kws = hist_kws_dict)
        ax.set_title("Histogram of Quality of Apples")
        ax.set_xlabel("Apple Quality")
        ax.set_ylabel("Count")
        ax_other = ax
        return ax_other

 
    def plotWeightDistn(self,ax_other = None):
        hist_kws_dict={"histtype": "bar", "linewidth": 1.5,
                  "alpha": 1, "color": "g","edgecolor":"black"}
        y = np.hstack(self.W_list)
        ax = sns.distplot(y,kde = False, hist_kws = hist_kws_dict)
        ax.set_title("Histogram of Weight of Apples")
        ax.set_xlabel("Apple Weight (g)")
        ax.set_ylabel("Count")
        ax_other = ax
        return ax_other


    
    def plotAmountDistn(self,ax_other = None):
        hist_kws_dict={"histtype": "bar", "linewidth": 1.5,
                  "alpha": 1, "color": "g","edgecolor":"black"}
        y = np.hstack(self.A)
        ax = sns.distplot(y,kde = False, hist_kws = hist_kws_dict)
        ax.set_title("Histogram of Apple Amount")
        ax.set_xlabel("Apples Amount")
        ax.set_ylabel("Count")
        ax_other = ax
        return ax_other

    
    def plotApplePriceDistn(self,ax_other = None):
        prices = np.hstack(self.P_list)
        prices = np.sort(prices)
        ax = sns.countplot(x = prices)
        ax.set_title("Countplot of Apple Price")
        ax.set_xlabel("Apple Price ($)")
        ax.set_ylabel("Count")
        ax_other = ax
        return ax_other

### OrchardSim Visualization Widget

In [4]:
@interact(m = widgets.IntSlider(min=10,max=1000,step=10,value=100,continuous_update=False),
         lam = widgets.IntSlider(min = 50, max = 400, step = 10, value = 211,continuous_update=False),
         r1 = widgets.FloatSlider(min = 0,max = 5,step = 0.5,value = 5,continuous_update=False),
         r2 =widgets.FloatSlider(min = 0,max = 5,step = 0.5,value = 3,continuous_update=False),
         r3 = widgets.FloatSlider(min = 0,max = 5,step = 0.5,value = 1.50,continuous_update=False),
         mu = widgets.FloatSlider(min = 130,max = 180,step = 10,value = 150,continuous_update=False),
         sigma2 = widgets.FloatSlider(min =700,max = 1200,step = 50,value = 900,continuous_update=False),
         alpha = widgets.IntSlider(min = 1,max = 100,step = 1,value = 5,continuous_update=False),
         beta = widgets.IntSlider(min = 1,max = 100,step = 1,value = 5,continuous_update=False))

def expZ(m,lam,r1,r2,r3,mu,sigma2,alpha,beta):


    from matplotlib import pyplot as plt
    nscen = 100
    trees = m
    amt_param = lam
    q_alpha = alpha
    q_beta = beta
    w_mean = mu
    w_var = sigma2
    anls = OrchardSim(nscen,trees,amt_param,q_alpha,q_beta,
                              w_mean,w_var,)
    anls.simOrchardGrowth()


    plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')

    plt.tight_layout()

    ax1 = plt.subplot(311)
    anls.plotRevenueDistn(ax1)

    ax2 = plt.subplot(323)
    anls.plotWeightDistn(ax2)

    ax3 = plt.subplot(324)
    anls.plotAmountDistn(ax3)

    ax4 = plt.subplot(325)
    anls.plotQualityDistn(ax4)

    ax5 = plt.subplot(326)
    anls.plotApplePriceDistn(ax5)



interactive(children=(IntSlider(value=100, continuous_update=False, description='m', max=1000, min=10, step=10…